In [139]:
from langchain_community.chat_models import ChatOllama
from typing import TypedDict
import json 
from langchain.messages import SystemMessage, HumanMessage
import os
from langgraph.graph import StateGraph, END
import ast

In [140]:
class AgentState(TypedDict):
    query: str 
    answer: str 
    feedback: str 
    attempt_count: int
    solved: bool

In [141]:
class LongTermMemory:
    def __init__(self, filepath = "agent_memory10.json"):
        self.filepath = filepath 
        if not os.path.exists(filepath):
            with open(filepath, "w", encoding = 'utf-8') as f:
                json.dump([], f)

    def load_lessons(self):
        with open(self.filepath, "r", encoding = 'utf-8') as f:
            return json.load(f)
    
    def save_lesson(self, lesson):
        lessons = self.load_lessons()
        if lesson not in lessons:
            with open(self.filepath, "w", encoding = 'utf-8') as f:
                json.dump(lessons, f, ensure_ascii = False, indent = 2)
            print(f"New lesson is saved in long term memory: {lesson}")

In [142]:
memory_system = LongTermMemory()

In [143]:
def code_generator_node(state: AgentState):
    query = state["query"]
    feedback = state.get("feedback", "")
    attempt_count = state["attempt_count"]

    past_lessons = memory_system.load_lessons()
    
    sys_msg = """
    you are a professional python coding assistant experienced in python coding professionally based on the user query.
    """ 

    hmn_msg = f"""
    Write a python code for user query:
    user_query: {query}
    """
    
    if past_lessons:
        hmn_msg += "\nRules and lessons you learned from past experiences (consider them for generating the code:\n)"
        for lesson in past_lessons:
            hmn_msg += f"- {lesson}\n"

    if feedback:
        hmn_msg += f"Your last try in this task faced with this problems: {feedback}\ngenerate an improved code."

    llm = ChatOllama(
        model = "codellama",
        temperature = 0
    )

    message = [
        SystemMessage(content = sys_msg),
        HumanMessage(content = hmn_msg)
    ]

    response = llm.invoke(message).content 

    print(f"Code Number {attempt_count}:\n{response}\n")

    return {
        "answer": response,
        "attempt_count": attempt_count + 1
    }

In [ ]:
def validator_node(state: AgentState):
    query = state["query"]
    answer = state["answer"]
    attempt_count = state["attempt_count"]
    
    print(f"\n---Try number: {attempt_count} ---")

    feedback = ""
    solved = False 

    sys_msg = """
    You are a professional python code validator based on the query. 
    validate if the code corresponds to the query or not. give an integer score between 0 to 100. 0 means code absolutely not corresponds to the query and 100 means absolutely corresponds to the query.
    corresponding to the query means that when the code is running, satisfy the user query without any error.
    your output should just an integer score between 0 to 100. avoid any explaination. 
    """

    hmn_msg = f"""
    validate the code based on the user query:

    user_query: {query}

    code: {answer}

    just give an integer score between 0 to 100 without any explaination.
    """

    message = [
        SystemMessage(content = sys_msg),
        HumanMessage(content = hmn_msg)
    ]


    llm = ChatOllama(
        model = "codellama",
        temperature = 0
    )

    response = llm.invoke(message).content
    print(response)

    llm2 = ChatOllama(
        model = "llama3",
        temperature = 0
    )

    pr = f"""
    a code validator has gave a report and score between 0 to 100 to the python code. 
    just output an integer score that code has been gave. avoid any explanation.  

    report: {response} 
    """

    response2 = llm2.invoke(pr).content

    score = ast.literal_eval(response2)
    print(f"score: {score}")

    try:
        if score < 80:
            prompt = """
            You are a professional debugger python code assistant. 
            the python code that is wrote based on the query {query} has got {score} score from 100 for how much corresponds to the query.
            check and analysis the code has wrote and determine why the code has got {score} score from 100.
            then specify which part of code should change to get higher score and more accurate code based on the query.
            """

            response2 = llm.invoke(prompt).content 
            
            raise ValueError(f"The code is not accurately corresponds to the query. The code that you has wrote has this problems: {response2}")

        print("Success! The code is highly corresponds to the user query.")
        solved = True

    except Exception as e:
        feedback = str(e)

    return {
        "solved": solved,
        "feedback": feedback
    }

In [ ]:
def reflector_node(state: AgentState):
    feedback = state["feedback"]
    query = state["query"]
    answer = state["answer"]

    reflection_prompt = (
    f"Based on the user query, the following code has been generated:\n"
    "user query: {query}"
    "generated code: {answer}"
    "the generated code has following problem:"
    "problem: {feedback}"
    "extract a short rule (one line) that usefull in the future."
    "for example, tell which points should consider to get more accurate code based on the user query."
    )

    llm = ChatOllama(
        model = "llama3",
        temperature = 0
    )

    lesson = llm.invoke([HumanMessage(content = reflection_prompt)]).content


    memory_system.save_lesson(lesson)

    return {}

In [146]:
def router(state: AgentState):
    if state["solved"]:
        return END 
    if state["attempt_count"] > 3:
        return END 
    return "reflector"

In [147]:
workflow = StateGraph(AgentState)

workflow.add_node("generator", code_generator_node)
workflow.add_node("validator", validator_node)
workflow.add_node("reflector", reflector_node)

workflow.set_entry_point("generator")
workflow.add_edge("generator", "validator")
workflow.add_conditional_edges(
    "validator",
    router, {
        "reflector": "reflector",
        END: END
    }
)
workflow.add_edge("reflector", "generator")

app = workflow.compile()

In [148]:
def run_agent(query: str):
    initial_state = {
        "query": query,
        "answer": "",
        "feedback": "",
        "attempt_count": 0,
        "solved": False
    }

    final_state = app.invoke(initial_state)

    print("\n======= Final Result =======")
    print(f"\nSolved: {final_state.get('solved')}")
    print(f"\nAttempts: {final_state.get('attempt_count')}")
    print(f"\nLast feedback: {final_state.get('feedback')}")

    print("\n======= Long Term Memory =======")
    lessons = memory_system.load_lessons()
    for i, lesson in enumerate(lessons, 1):
        print(f"{i}. {lesson}")

    return final_state

In [149]:
response = run_agent("How to write an english teacher agent using langgraph?")

Code Number 0:

To create an English teacher agent using LangGraph, you can follow these steps:

1. Install the necessary dependencies:
```
pip install langgraph
```
2. Import the necessary libraries:
```python
import langgraph as lg
from langgraph import Graph
```
3. Create a graph object:
```python
graph = Graph()
```
4. Add nodes to the graph representing different parts of speech:
```python
# Nodes for nouns, verbs, adjectives, and adverbs
noun_node = lg.Node("Noun")
verb_node = lg.Node("Verb")
adjective_node = lg.Node("Adjective")
adverb_node = lg.Node("Adverb")
```
5. Add edges between the nodes to represent grammatical relationships:
```python
# Edges for subject-verb agreement
graph.add_edge(noun_node, verb_node)
graph.add_edge(adjective_node, verb_node)
graph.add_edge(adverb_node, verb_node)

# Edges for object-verb agreement
graph.add_edge(noun_node, verb_node)
graph.add_edge(adjective_node, verb_node)
graph.add_edge(adverb_node, verb_node)
```
6. Add nodes and edges to repre

In [138]:
if os.path.exists("agent_memory10.json"):
    os.remove("agent_memory10.json")
    print("Memory is cleared. Now you can run from task 1.")

Memory is cleared. Now you can run from task 1.
